<a href="https://colab.research.google.com/github/tanishi22/FYP/blob/main/Code/Feature_Merging/extrinsic_feature_processing_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Gene IDs

In [ ]:
# Load essential/nonessential gene IDs
essential_genes = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/essential_genes.txt", header=None)[0].tolist()
nonessential_genes = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/nonessential_genes.txt", header=None)[0].tolist()

In [ ]:
essential_ids = pd.DataFrame({'FBgn_ID': essential_genes})
nonessential_ids = pd.DataFrame({'FBgn_ID': nonessential_genes})

In [ ]:
# Load essential/non-essential genes from batches (created for easier batch download of data for extrinsic feature extraction and engineering)
EG1_df = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/EG_1_isoforms.csv", header=None)
EG2_df = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/EG_2_isoforms.csv", header=None)

NEG1_df = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/NEG_1_isoforms.csv", header=None)
NEG2_df = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/NEG_2_isoforms.csv", header=None)
NEG3_df = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/NEG_3_isoforms.csv", header=None)
NEG4_df = pd.read_csv("/content/drive/My Drive/ICL/FYP/Data/labels/NEG_4_isoforms.csv", header=None)

In [ ]:
# Process data frames to create FBgn_ID - isoform maps
all_genes_df = pd.concat([EG1_df, EG2_df, NEG1_df, NEG2_df, NEG3_df, NEG4_df]) # Merge all

all_genes_df.columns = all_genes_df.iloc[0]
all_genes_df = all_genes_df[1:].reset_index(drop=True)
all_genes_df.columns.name = None  # <-- add this line!

# then the rest as before
all_genes_df.drop(all_genes_df.columns[[0, 2, 4]], axis=1, inplace=True)
all_genes_df.rename(columns={'Longest_Isoform_ID': 'ensembl_peptide_id'}, inplace=True)

all_genes_dedup = all_genes_df.drop_duplicates(subset=['FBgn_ID', 'ensembl_peptide_id'])

In [ ]:
# Check how many times each key appears
dup_all_genes = all_genes_df.duplicated(subset=['FBgn_ID'], keep=False)

print(f"Duplicate rows in all_genes_df: {dup_all_genes.sum()}")

Duplicate rows in all_genes_df: 5


In [ ]:
all_genes_dedup = all_genes_df.drop_duplicates(subset = ['FBgn_ID', 'ensembl_peptide_id'])

,FBgn_ID,ensembl_peptide_id
0,FBgn0010612,FBpp0297136
1,FBgn0003517,FBpp0308665
2,FBgn0015376,FBpp0077230
3,FBgn0004868,FBpp0079458
4,FBgn0013733,FBpp0086747
...,...,...
5845,FBgn0052058,FBpp0076022
5846,FBgn0267489,FBpp0312368
5847,FBgn0065109,FBpp0100162
5848,FBgn0267429,FBpp0312363


## Load and organise extrinsic features

# **Transcriptomics and Proteomics**
Tissue and developmental stage-level gene expression summary metrics.
PPI network topology features.

**Transcriptomics**

In [ ]:
# Load data
EG_flyatlas = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/transcriptomics/EG_flyatlas_features.csv')
NEG_flyatlas = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/transcriptomics/NEG_flyatlas_features.csv')

EG_med = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/transcriptomics/EG_med_features.csv')
NEG_med = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/transcriptomics/NEG_med_features.csv')

EG_met = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/transcriptomics/EG_met_features.csv')
NEG_met = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/transcriptomics/NEG_met_features.csv')

# concatenate all features at the level of essentiality
EG_transcriptomics = EG_flyatlas.merge(EG_med, on = 'Gene_ID', how = 'left')
EG_transcriptomics = EG_transcriptomics.merge(EG_met, on = 'Gene_ID', how = 'left')

NEG_transcriptomics = NEG_flyatlas.merge(NEG_med, on = 'Gene_ID', how = 'left')
NEG_transcriptomics = NEG_transcriptomics.merge(NEG_met, on = 'Gene_ID', how = 'left')

# concatenate all
transcriptomics = pd.concat([EG_transcriptomics, NEG_transcriptomics], ignore_index=True)
transcriptomics.rename(columns={transcriptomics.columns[0]: 'FBgn_ID'}, inplace=True)

# 5840 rows x 31 columns - needs padding with missing gene IDs
transcriptomics_padded = pd.merge(
    all_genes_dedup[['FBgn_ID']],
    transcriptomics,
    on = 'FBgn_ID',
    how = 'left'
)

transcriptomics_padded

,FBgn_ID,fa_mean_expression,fa_median_expression,fa_max_expression,fa_tissue_with_max_expression,fa_expression_variance,fa_expression_sd,fa_expression_breadth,fa_Tau_Score,fa_Shannon_Entropy,...,met_mean_expression,met_median_expression,met_max_expression,met_tissue_with_max_expression,met_expression_variance,met_expression_sd,met_expression_breadth,met_Tau_Score,met_Shannon_Entropy,met_Shannon_Entropy_Specificity
0,FBgn0000008,2.256404,2.000000,5.285402,RNA-Seq_Profile_FlyAtlas2_Adult_Female_Virgin_...,1.380451,1.174926,35,0.681133,3.465371,...,3.146540,3.321928,6.087463,mE_mRNA_WPP_fat,1.282540,1.132493,27.0,0.578073,4.021268,0.101135
1,FBgn0000014,1.641483,1.000000,6.686501,RNA-Seq_Profile_FlyAtlas2_Adult_Female_Virgin_...,3.246864,1.801906,21,0.732551,3.068724,...,1.681604,1.000000,5.426265,mE_mRNA_L3_Wand_carcass,3.253159,1.803651,14.0,0.577834,2.701797,0.959957
2,FBgn0000015,1.435746,1.000000,5.523562,RNA-Seq_Profile_FlyAtlas2_L3_Trachea,2.827941,1.681648,19,0.675149,2.819795,...,0.672133,1.000000,3.000000,mE_mRNA_A_MateM_4d_acc_gland,0.618373,0.786367,4.0,0.850272,1.638784,1.054449
3,FBgn0000017,3.314307,3.000000,5.209453,RNA-Seq_Profile_FlyAtlas2_L3_CNS,1.002952,1.001475,42,0.450212,4.016284,...,2.815434,3.000000,5.000000,mE_mRNA_L3_CNS,1.200604,1.095721,26.0,0.546142,3.810928,0.065430
4,FBgn0000018,3.079274,3.000000,5.209453,RNA-Seq_Profile_FlyAtlas2_Adult_Female_Ovary,0.365334,0.604429,43,0.737718,3.122235,...,1.735130,1.584963,3.584963,mE_mRNA_L3_Wand_imag_disc,0.523804,0.723743,22.0,0.691640,2.500031,0.108764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5835,FBgn0289678,0.574727,0.000000,7.918863,RNA-Seq_Profile_FlyAtlas2_Adult_Male_Testis,2.241798,1.497263,6,0.787345,1.258360,...,0.944629,0.000000,7.442943,mE_mRNA_A_MateM_4d_testis,3.567675,1.888829,5.0,0.682784,1.601479,1.990774
5836,FBgn0289743,2.540161,2.321928,6.375039,RNA-Seq_Profile_FlyAtlas2_Adult_Male_Malpighia...,2.990574,1.729328,30,0.651278,3.876749,...,1.482568,2.000000,4.392317,mE_mRNA_A_MateM_20d_head,1.930931,1.389580,16.0,0.549250,2.646293,0.813368
5837,FBgn0289744,3.005701,3.000000,4.954196,RNA-Seq_Profile_FlyAtlas2_Adult_Female_Ovary,0.375780,0.613009,43,0.659569,3.285448,...,2.467004,2.584963,4.392317,mE_mRNA_A_VirF_4d_ovary,0.606976,0.779087,26.0,0.567551,3.046625,0.051489
5838,FBgn0289745,1.775716,1.000000,6.832890,RNA-Seq_Profile_FlyAtlas2_Adult_Male_Testis,2.561208,1.600377,20,0.786425,3.067150,...,1.503996,1.000000,4.247928,mE_mRNA_WPP_saliv,1.237705,1.112522,14.0,0.748354,2.678897,0.453043


**Protein-protein interaction network topology**

In [ ]:
# Load data
EG_PPIn = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/PPIns/PPIn_essential.csv')
NEG_PPIn = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/PPIns/PPIn_nonessential.csv')

# Note, not all genes had PPI information available, so need to bind these gene-level rows in with NA values across the following features
# PPIn features were extracted with essential and non_essential IDs
# Need to merge existing feature data onto full gene list to add missing genes with NaNs
EG_PPIn_full = essential_ids.merge(EG_PPIn, on='FBgn_ID', how='left')
NEG_PPIn_full = nonessential_ids.merge(NEG_PPIn, on='FBgn_ID', how='left')

# Merge EG_PPIn_full and NEG_PPIn_full into a single PPIn matrix
PPIn_full = pd.concat([EG_PPIn_full, NEG_PPIn_full], ignore_index=True)
PPIn_full.head()

,FBgn_ID,Degree,Degree Centrality,Closeness Centrality,Betweenness Centrality,Eigenvector Centrality,Average Neighbor Degree,Clustering Coefficient,Triangle Count,Core Number,Louvain Community
0,FBgn0000008,1.0,0.000158,0.218239,0.000000,2.880723e-05,42.000000,0.000000,0.0,1.0,17.0
1,FBgn0000014,163.0,0.025685,0.315579,0.012630,1.000753e-03,23.067485,0.038703,511.0,12.0,12.0
2,FBgn0000015,41.0,0.006461,0.273769,0.000486,1.054210e-04,26.878049,0.135366,111.0,11.0,12.0
3,FBgn0000017,21.0,0.003309,0.267418,0.001300,6.153944e-05,23.000000,0.095238,20.0,8.0,32.0
4,FBgn0000018,1.0,0.000158,0.195339,0.000000,9.168425e-07,9.000000,0.000000,0.0,1.0,17.0


# **Evolutionary Conservation**
Paralogs

In [ ]:
# Load data and merge
EG_paralogs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/paralogs/EG_paralog_features.csv')
NEG_paralogs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/paralogs/NEG_paralog_features.csv')
paralogs_full = pd.concat([EG_paralogs, NEG_paralogs]) # n = 5845 genes so no need for padding
paralogs_full

,FBgn_ID,Num_Paralogs,paralogs_presence,para_mean_diopt_score,para_median_diopt_score,para_max_diopt_score,para_min_diopt_score,paralog_same_arm,avg_distance_same_arm
0,FBgn0000008,28,1,1.178571,1.0,3,1,1.0,6.104371e+06
1,FBgn0000014,82,1,2.292683,2.0,6,1,1.0,6.505879e+06
2,FBgn0000015,86,1,1.581395,1.0,4,1,1.0,6.650291e+06
3,FBgn0000017,60,1,1.900000,1.0,7,1,1.0,8.694273e+06
4,FBgn0000018,0,0,0.000000,0.0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3759,FBgn0289678,0,0,0.000000,0.0,0,0,NaN,NaN
3760,FBgn0289743,69,1,5.942029,7.0,12,1,1.0,5.942577e+06
3761,FBgn0289744,12,1,4.333333,4.0,10,2,1.0,2.039530e+06
3762,FBgn0289745,35,1,6.028571,7.0,9,1,1.0,1.451566e+07


Orthologs

In [ ]:
# Load orthologs data (downloaded in lots of batches + already pre-processed) and merge
EG_1_b1_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/EG1_b1_orthology.csv')
EG_1_b2_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/EG1_b2_orthology.csv')

EG_2_b1_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/EG2_b1_orthology.csv')
EG_2_b2_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/EG2_b2_orthology.csv')
EG_2_b3_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/EG2_b3_orthology.csv')

NEG_1_b1_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG1_b1_orthology.csv')
NEG_1_b2_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG1_b2_orthology.csv')
NEG_2_b1_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG2_b1_orthology.csv')
NEG_2_b2_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG2_b2_orthology.csv')
NEG_3_b1_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG3_b1_orthology.csv')
NEG_3_b2_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG3_b2_orthology.csv')
NEG_4_b1_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG4_b1_orthology.csv')
NEG_4_b2_orthologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/processed/NEG4_b2_orthology.csv')

orthologs_full = pd.concat([EG_1_b1_orthologs, EG_1_b2_orthologs, EG_2_b1_orthologs, EG_2_b2_orthologs,
                            NEG_1_b1_orthologs, NEG_1_b2_orthologs, NEG_2_b1_orthologs, NEG_2_b2_orthologs,
                            NEG_3_b1_orthologs, NEG_3_b2_orthologs, NEG_4_b1_orthologs, NEG_4_b2_orthologs])

orthologs_full # 5397 rows x 31 columns - needs padding to account for genes with no orthologs, so will be 0 across all these numeric features

In [ ]:
# Create padded ortholog set by performing an outer join to preserve all FBgn_IDs from paralogs_full
orthologs_padded = pd.merge(
    paralogs_full[['FBgn_ID']],   # all genes
    orthologs_full,               # only genes with orthologs
    on='FBgn_ID',
    how='left'
)

# Fill missing values with 0
orthologs_padded.fillna(0, inplace=True)

Human Disease Homologs

In [ ]:
# Load data and merge
EG1_disease_homologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/EG1_disease_homologs.csv')
EG2_disease_homologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/EG2_disease_homologs.csv')

NEG1_disease_homologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/NEG1_disease_homologs.csv')
NEG2_disease_homologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/NEG2_disease_homologs.csv')
NEG3_disease_homologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/NEG3_disease_homologs.csv')
NEG4_disease_homologs = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/diopt/NEG4_disease_homologs.csv')

disease_homologs_full = pd.concat([EG1_disease_homologs, EG2_disease_homologs, NEG1_disease_homologs, NEG2_disease_homologs,
                                   NEG3_disease_homologs, NEG4_disease_homologs])

disease_homologs_full # 5845 rows x 2 columns, all genes present, no need for padding

,FBgn_ID,Disease_Homolog_Count
0,FBgn0000008,1
1,FBgn0000014,18
2,FBgn0000015,13
3,FBgn0000017,10
4,FBgn0000018,1
...,...,...
759,FBgn0266710,0
760,FBgn0261614,0
761,FBgn0051438,0
762,FBgn0053527,0


In [ ]:
# Merge all evolution-related features
evolution_features = paralogs_full.merge(orthologs_padded, on = 'FBgn_ID', how = 'left') # merge complete datasets first
evolution_features = evolution_features.merge(disease_homologs_full, on = 'FBgn_ID', how = 'left' )
evolution_features

,FBgn_ID,Num_Paralogs,paralogs_presence,para_mean_diopt_score,para_median_diopt_score,para_max_diopt_score,para_min_diopt_score,paralog_same_arm,avg_distance_same_arm,Ortholog_Count,...,Fly_Mouse_AvgDIOPT,Fly_Rat_AvgDIOPT,Fly_Thale cress_AvgDIOPT,Fly_Western clawed frog_AvgDIOPT,Fly_Worm_AvgDIOPT,Fly_Yeast_AvgDIOPT,Fly_Zebrafish_AvgDIOPT,High_Rank_Ortholog_Count,high_ranking_ortholog_presence,Disease_Homolog_Count
0,FBgn0000008,28,1,1.178571,1.0,3,1,1.0,6.104371e+06,18.0,...,3.600000,4.000000,0.000000,3.0000,2.000000,0.0,3.000000,6.0,1.0,1.0
1,FBgn0000014,82,1,2.292683,2.0,6,1,1.0,6.505879e+06,136.0,...,3.880000,3.785714,3.000000,3.0000,3.571429,2.0,3.846154,3.0,1.0,18.0
2,FBgn0000015,86,1,1.581395,1.0,4,1,1.0,6.650291e+06,86.0,...,4.333333,4.600000,3.000000,3.6875,5.500000,2.0,4.625000,13.0,1.0,13.0
3,FBgn0000017,60,1,1.900000,1.0,7,1,1.0,8.694273e+06,148.0,...,3.913043,3.578947,3.833333,4.2000,3.708333,0.0,3.782609,9.0,1.0,10.0
4,FBgn0000018,0,0,0.000000,0.0,0,0,NaN,NaN,7.0,...,14.000000,14.000000,13.000000,12.0000,0.000000,0.0,14.000000,7.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5840,FBgn0289678,0,0,0.000000,0.0,0,0,NaN,NaN,0.0,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.0
5841,FBgn0289743,69,1,5.942029,7.0,12,1,1.0,5.942577e+06,0.0,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,6.0
5842,FBgn0289744,12,1,4.333333,4.0,10,2,1.0,2.039530e+06,0.0,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,2.0
5843,FBgn0289745,35,1,6.028571,7.0,9,1,1.0,1.451566e+07,0.0,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,8.0


# **Gene Ontology**

In [ ]:
# Load all datasets
EG1_ontology = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/ontology/EG_1_ontology.csv')
EG2_ontology = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/ontology/EG_2_ontology.csv')

NEG1_ontology = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/ontology/NEG_1_ontology.csv')
NEG2_ontology = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/ontology/NEG_2_ontology.csv')
NEG3_ontology = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/ontology/NEG_3_ontology.csv')
NEG4_ontology = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/ontology/NEG_4_ontology.csv')

# Merge into complete ontology dataframe - Gene ID is ok
ontology = pd.concat([EG1_ontology, EG2_ontology, NEG1_ontology, NEG2_ontology, NEG3_ontology], ignore_index=True)
ontology

# 4682 x 5 columns
# Can see several missing genes, so need to make a padded version with NA terms
# Create padded ontology set by performing an outer join to preserve all FBgn_IDs from paralogs_full (which has all gene IDs)
ontology_padded = pd.merge(
    paralogs_full[['FBgn_ID']],   # all genes
    ontology,               # only genes with orthologs
    on='FBgn_ID',
    how='left'
)

# Fill missing values with NA
ontology_padded.fillna(np.nan, inplace=True)
ontology_padded

,FBgn_ID,biological_process,cellular_component,molecular_function,goslim_terms
0,FBgn0000008,1.0,2.0,2.0,anatomical structure development
1,FBgn0000014,260.0,40.0,70.0,organelle; nucleus; DNA binding; transcription...
2,FBgn0000015,234.0,18.0,45.0,organelle; nucleus; DNA binding; anatomical st...
3,FBgn0000017,35.0,0.0,25.0,anatomical structure development; catalytic ac...
4,FBgn0000018,28.0,21.0,21.0,organelle; chromosome; nucleus; regulation of ...
...,...,...,...,...,...
5840,FBgn0289678,NaN,NaN,NaN,NaN
5841,FBgn0289743,NaN,NaN,NaN,NaN
5842,FBgn0289744,NaN,NaN,NaN,NaN
5843,FBgn0289745,NaN,NaN,NaN,NaN


In [ ]:
ontology_padded['goslim_terms']

,goslim_terms
0,anatomical structure development
1,organelle; nucleus; DNA binding; transcription...
2,organelle; nucleus; DNA binding; anatomical st...
3,anatomical structure development; catalytic ac...
4,organelle; chromosome; nucleus; regulation of ...
...,...
5840,NaN
5841,NaN
5842,NaN
5843,NaN


In [ ]:
# Need to re-format multilabel categorical variables like goslim_terms to prepare for MultiLabelBinarizer in the final ML pipeline
def parse_multilabel(val):
    if pd.isna(val):
        return []
    return [loc.strip() for loc in val.split(';')]

ontology_padded['goslim_terms'] = ontology_padded['goslim_terms'].apply(parse_multilabel)
ontology_padded['goslim_terms']

,goslim_terms
0,[anatomical structure development]
1,"[organelle, nucleus, DNA binding, transcriptio..."
2,"[organelle, nucleus, DNA binding, anatomical s..."
3,"[anatomical structure development, catalytic a..."
4,"[organelle, chromosome, nucleus, regulation of..."
...,...
5840,[]
5841,[]
5842,[]
5843,[]


# **Subcellular Localisation Features**
SignalP, DeepLoc, and DeepTMHMM features.

In [ ]:
# Load SignalP data
EG1_signalp = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/signalp/EG1_signalp.csv')
EG2_signalp = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/signalp/EG2_signalp.csv')

NEG1_signalp = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/signalp/NEG1_signalp.csv')
NEG2_signalp = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/signalp/NEG2_signalp.csv')
NEG3_signalp = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/signalp/NEG3_signalp.csv')
NEG4_signalp = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/signalp/NEG4_signalp.csv')

# Concatenate all rows into one complete signalp matrix
signalp = pd.concat([EG1_signalp, EG2_signalp, NEG1_signalp, NEG2_signalp, NEG3_signalp, NEG4_signalp])
signalp.head()

# all signalp rows present, but organised at level of ensembl_peptide_id, so will need to map to gene IDs using isoform maps
signalp_full = pd.merge(
    all_genes_df[['FBgn_ID', 'ensembl_peptide_id']],
    signalp,
    on = 'ensembl_peptide_id',
    how = 'left'
)

signalp_full.head()

,FBgn_ID,ensembl_peptide_id,signalp_prediction,signalp_other,signalp_sp_sec_spi
0,FBgn0010612,FBpp0297136,OTHER,1.0,0.0
1,FBgn0003517,FBpp0308665,OTHER,1.0,0.0
2,FBgn0015376,FBpp0077230,OTHER,1.0,0.0
3,FBgn0004868,FBpp0079458,OTHER,1.0,0.0
4,FBgn0013733,FBpp0086747,OTHER,1.0,0.0


In [ ]:
# Load Deeploc data - done in multiple batches
EG1_b1_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/EG1_b1_deeploc.csv')
EG1_b2_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/EG1_b2_deeploc.csv')

EG2_b1_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/EG2_b1_deeploc.csv')
EG2_b2_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/EG2_b2_deeploc.csv')
EG2_b3_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/EG2_b3_deeploc.csv')

NEG1_b1_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG1_b1_deeploc.csv')
NEG1_b2_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG1_b2_deeploc.csv')
NEG2_b1_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG2_b1_deeploc.csv')
NEG2_b2_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG2_b2_deeploc.csv')
NEG3_b1_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG3_b1_deeploc.csv')
NEG3_b2_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG3_b2_deeploc.csv')
NEG4_b1_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG4_b1_deeploc.csv')
NEG4_b2_deeploc = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeploc/NEG4_b2_deeploc.csv')


# Concatenate all rows into one complete signalp matrix
deeploc = pd.concat([EG1_b1_deeploc, EG1_b2_deeploc, EG2_b1_deeploc, EG2_b2_deeploc, EG2_b3_deeploc, NEG1_b1_deeploc, NEG1_b2_deeploc,
                           NEG2_b1_deeploc, NEG2_b2_deeploc, NEG3_b1_deeploc, NEG3_b2_deeploc, NEG4_b1_deeploc, NEG4_b2_deeploc])
deeploc.head()


 # not all rows present so needs padding - missing rows because DeepLoc can only handle protein sequences 10 < length < 6000,
 # so some proteinswere filtered out.

 # organised at level of ensembl_peptide_id, so will need to map to gene IDs using isoform maps
deeploc_full = pd.merge(
    signalp_full[['FBgn_ID', 'ensembl_peptide_id']],
    deeploc,
    on = 'ensembl_peptide_id',
    how = 'left'
)

deeploc_full.head()

,FBgn_ID,ensembl_peptide_id,Localizations,Signals,p_Cytoplasm,p_Nucleus,p_Extracellular,p_Cell_membrane,p_Mitochondrion,p_Plastid,p_Endoplasmic_reticulum,p_Lysosome_Vacuole,p_Golgi_apparatus,p_Peroxisome
0,FBgn0010612,FBpp0297136,Mitochondrion,Mitochondrial transit peptide,0.1323,0.0443,0.0571,0.1183,0.9397,0.0167,0.2031,0.2004,0.0893,0.1596
1,FBgn0003517,FBpp0308665,Cytoplasm,NaN,0.7541,0.2989,0.0837,0.0642,0.1157,0.0232,0.0699,0.1003,0.1474,0.0124
2,FBgn0015376,FBpp0077230,Nucleus,Nuclear localization signal; Nuclear export si...,0.2943,0.8807,0.0054,0.0327,0.0827,0.0039,0.0349,0.0489,0.0437,0.0203
3,FBgn0004868,FBpp0079458,Cytoplasm,Nuclear export signal,0.8305,0.3559,0.0271,0.0776,0.2353,0.0398,0.2976,0.0672,0.2808,0.0164
4,FBgn0013733,FBpp0086747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Need to re-format multilabel categorical variables like Localizations and Signal to prepare for MultiLabelBinarizer in the final ML pipeline
deeploc_full['Localizations']

,Localizations
0,Mitochondrion
1,Cytoplasm
2,Nucleus
3,Cytoplasm
4,NaN
...,...
5845,Cell membrane; Lysosome/Vacuole
5846,Cell membrane
5847,Cell membrane
5848,Cell membrane; Lysosome/Vacuole


In [ ]:
deeploc_full['Localizations'] = deeploc_full['Localizations'].apply(parse_multilabel)
deeploc_full['Signals'] = deeploc_full['Signals'].apply(parse_multilabel)

In [ ]:
# Load DeepTMHMM results - done in even more batches!!!
EG1_b1_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG1_b1_TMRs.csv')
EG1_b2_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG1_b2_TMRs.csv')
EG1_b3_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG1_b3_TMRs.csv')
EG1_b4_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG1_b4_TMRs.csv')

EG2_b1_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG2_b1_TMRs.csv')
EG2_b2_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG2_b2_TMRs.csv')
EG2_b3_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG2_b3_TMRs.csv')
EG2_b4_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG2_b4_TMRs.csv')
EG2_b5_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/EG2_b5_TMRs.csv')

NEG1_b1_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG1_b1_TMRs.csv')
NEG1_b2_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG1_b2_TMRs.csv')
NEG1_b3_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG1_b3_TMRs.csv')
NEG1_b4_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG1_b4_TMRs.csv')

NEG2_b1_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG2_b1_TMRs.csv')
NEG2_b2_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG2_b2_TMRs.csv')
NEG2_b3_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG2_b3_TMRs.csv')
NEG2_b4_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG2_b4_TMRs.csv')

NEG3_b1_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG3_b1_TMRs.csv')
NEG3_b2_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG3_b2_TMRs.csv')
NEG3_b3_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG3_b3_TMRs.csv')
NEG3_b4_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG3_b4_TMRs.csv')

NEG4_b1_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG4_b1_TMRs.csv')
NEG4_b2_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG4_b2_TMRs.csv')
NEG4_b3_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG4_b3_TMRs.csv')
NEG4_b4_deeptmhmm = pd.read_csv('/content/drive/My Drive/ICL/FYP/Data/extrinsic_features/deeptmhmm/NEG4_b4_TMRs.csv')


In [ ]:
# Merge all
deeptmhmm = pd.concat([EG1_b1_deeptmhmm,EG1_b2_deeptmhmm,EG1_b3_deeptmhmm, EG1_b4_deeptmhmm,
                            EG2_b1_deeptmhmm, EG2_b2_deeptmhmm, EG2_b3_deeptmhmm, EG2_b4_deeptmhmm, EG2_b5_deeptmhmm,
                            NEG1_b1_deeptmhmm, NEG1_b2_deeptmhmm, NEG1_b3_deeptmhmm, NEG1_b4_deeptmhmm,
                            NEG2_b1_deeptmhmm, NEG2_b2_deeptmhmm, NEG2_b3_deeptmhmm, NEG2_b4_deeptmhmm,
                            NEG3_b1_deeptmhmm, NEG3_b2_deeptmhmm, NEG3_b3_deeptmhmm, NEG3_b4_deeptmhmm,
                            NEG4_b1_deeptmhmm, NEG4_b2_deeptmhmm, NEG4_b3_deeptmhmm, NEG4_b4_deeptmhmm])

 # not all rows present so needs padding - missing rows because DeepTMHMM can only handle protein sequences 10 < length < 6000,
 # so some proteins were filtered out.

 # organised at level of ensembl_peptide_id, so will need to map to gene IDs using isoform maps
deeptmhmm_full = pd.merge(
    signalp_full[['FBgn_ID', 'ensembl_peptide_id']],
    deeptmhmm,
    on = 'ensembl_peptide_id',
    how = 'left'
)
deeptmhmm_full.head()

,FBgn_ID,ensembl_peptide_id,Num_TMRs,residues_inside,residues_TMhelix,residues_outside,residues_signal,residues_Beta sheet,residues_periplasm
0,FBgn0010612,FBpp0297136,1.0,68.0,21.0,10.0,0.0,0.0,0.0
1,FBgn0003517,FBpp0308665,0.0,270.0,0.0,0.0,0.0,0.0,0.0
2,FBgn0015376,FBpp0077230,0.0,993.0,0.0,0.0,0.0,0.0,0.0
3,FBgn0004868,FBpp0079458,0.0,443.0,0.0,0.0,0.0,0.0,0.0
4,FBgn0013733,FBpp0086747,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check how many times each key appears
dup_sig = signalp_full.duplicated(subset=['FBgn_ID', 'ensembl_peptide_id'], keep=False)
dup_deep = deeploc_full.duplicated(subset=['FBgn_ID', 'ensembl_peptide_id'], keep=False)
dup_deeptm = deeptmhmm_full.duplicated(subset=['FBgn_ID', 'ensembl_peptide_id'], keep=False)

print(f"Duplicate rows in signalp_full: {dup_sig.sum()}")
print(f"Duplicate rows in deeploc_full: {dup_deep.sum()}")
print(f"Duplicate rows in deeptmhmm_full: {dup_deeptm.sum()}")

Duplicate rows in signalp_full: 5
Duplicate rows in deeploc_full: 5
Duplicate rows in deeptmhmm_full: 5


In [ ]:
signalp_dedup = signalp_full.drop_duplicates(subset=['FBgn_ID', 'ensembl_peptide_id'])
deeploc_dedup = deeploc_full.drop_duplicates(subset=['FBgn_ID', 'ensembl_peptide_id'])
deeptmhmm_dedup = deeptmhmm_full.drop_duplicates(subset=['FBgn_ID', 'ensembl_peptide_id'])

subcellular_loc = pd.merge(
    signalp_dedup,
    deeploc_dedup,
    on=['FBgn_ID', 'ensembl_peptide_id'],
    how='inner'
)

subcellular_loc = pd.merge(
    subcellular_loc,
    deeptmhmm_dedup,
    on=['FBgn_ID', 'ensembl_peptide_id'],
    how='inner'
)

subcellular_loc.head()

,FBgn_ID,ensembl_peptide_id,signalp_prediction,signalp_other,signalp_sp_sec_spi,Localizations,Signals,p_Cytoplasm,p_Nucleus,p_Extracellular,...,p_Lysosome_Vacuole,p_Golgi_apparatus,p_Peroxisome,Num_TMRs,residues_inside,residues_TMhelix,residues_outside,residues_signal,residues_Beta sheet,residues_periplasm
0,FBgn0010612,FBpp0297136,OTHER,1.0,0.0,[Mitochondrion],[Mitochondrial transit peptide],0.1323,0.0443,0.0571,...,0.2004,0.0893,0.1596,1.0,68.0,21.0,10.0,0.0,0.0,0.0
1,FBgn0003517,FBpp0308665,OTHER,1.0,0.0,[Cytoplasm],[],0.7541,0.2989,0.0837,...,0.1003,0.1474,0.0124,0.0,270.0,0.0,0.0,0.0,0.0,0.0
2,FBgn0015376,FBpp0077230,OTHER,1.0,0.0,[Nucleus],"[Nuclear localization signal, Nuclear export s...",0.2943,0.8807,0.0054,...,0.0489,0.0437,0.0203,0.0,993.0,0.0,0.0,0.0,0.0,0.0
3,FBgn0004868,FBpp0079458,OTHER,1.0,0.0,[Cytoplasm],[Nuclear export signal],0.8305,0.3559,0.0271,...,0.0672,0.2808,0.0164,0.0,443.0,0.0,0.0,0.0,0.0,0.0
4,FBgn0013733,FBpp0086747,OTHER,1.0,0.0,[],[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Merge by gene ID

In [ ]:
# Merge by each df and check gene count in each step
extrinsic_df = ontology_padded.merge(subcellular_loc, on = 'FBgn_ID', how = 'inner')
extrinsic_df = extrinsic_df.merge(evolution_features, on = 'FBgn_ID', how = 'inner')
extrinsic_df = extrinsic_df.merge(transcriptomics_padded, on = 'FBgn_ID', how = 'inner')
extrinsic_df = extrinsic_df.merge(PPIn_full, on = 'FBgn_ID', how = 'inner')
extrinsic_df # total 107 features. will increase when goslim_terms and other categorical features are one-hot encoded

,FBgn_ID,biological_process,cellular_component,molecular_function,goslim_terms,ensembl_peptide_id,signalp_prediction,signalp_other,signalp_sp_sec_spi,Localizations,...,Degree,Degree Centrality,Closeness Centrality,Betweenness Centrality,Eigenvector Centrality,Average Neighbor Degree,Clustering Coefficient,Triangle Count,Core Number,Louvain Community
0,FBgn0000008,1.0,2.0,2.0,[anatomical structure development],FBpp0071678,OTHER,1.000000,0.000002,"[Cytoplasm, Cell membrane]",...,1.0,0.000158,0.218239,0.000000,2.880723e-05,42.000000,0.000000,0.0,1.0,17.0
1,FBgn0000014,260.0,40.0,70.0,"[organelle, nucleus, DNA binding, transcriptio...",FBpp0082829,OTHER,1.000000,0.000000,[Nucleus],...,163.0,0.025685,0.315579,0.012630,1.000753e-03,23.067485,0.038703,511.0,12.0,12.0
2,FBgn0000015,234.0,18.0,45.0,"[organelle, nucleus, DNA binding, anatomical s...",FBpp0082826,OTHER,1.000000,0.000002,[Nucleus],...,41.0,0.006461,0.273769,0.000486,1.054210e-04,26.878049,0.135366,111.0,11.0,12.0
3,FBgn0000017,35.0,0.0,25.0,"[anatomical structure development, catalytic a...",FBpp0303166,OTHER,1.000000,0.000001,"[Cytoplasm, Cell membrane]",...,21.0,0.003309,0.267418,0.001300,6.153944e-05,23.000000,0.095238,20.0,8.0,32.0
4,FBgn0000018,28.0,21.0,21.0,"[organelle, chromosome, nucleus, regulation of...",FBpp0079757,OTHER,1.000000,0.000000,[Nucleus],...,1.0,0.000158,0.195339,0.000000,9.168425e-07,9.000000,0.000000,0.0,1.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5834,FBgn0289579,NaN,NaN,NaN,[],FBpp0074906,OTHER,1.000000,0.000000,[Mitochondrion],...,9.0,0.001418,0.247857,0.000171,1.399735e-04,50.333333,0.388889,14.0,7.0,3.0
5835,FBgn0289678,NaN,NaN,NaN,[],FBpp0079449,OTHER,0.999913,0.000132,[Nucleus],...,1.0,0.000158,0.206459,0.000000,8.446426e-07,49.000000,0.000000,0.0,1.0,20.0
5836,FBgn0289743,NaN,NaN,NaN,[],FBpp0085466,OTHER,0.999905,0.000113,[Endoplasmic reticulum],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5837,FBgn0289744,NaN,NaN,NaN,[],FBpp0087120,OTHER,0.980650,0.019351,[Mitochondrion],...,5.0,0.000788,0.228193,0.000051,1.245923e-05,20.400000,0.400000,4.0,4.0,14.0


In [ ]:
# Assign label
def get_label(gene_id):
    if gene_id in essential_genes:
        return 1
    elif gene_id in nonessential_genes:
        return 0
    else:
        return None

extrinsic_df["label"] = extrinsic_df["FBgn_ID"].apply(get_label)

In [ ]:
# Numbers should be in line with essentiality annotations
# missing gene IDs here due to (1) ambiguous phentypes that were not filtered out (2) outdated FlyBase Gene IDs
print(extrinsic_df["label"].value_counts(dropna=False))

label
0    3763
1    2076
Name: count, dtype: int64


In [ ]:
extrinsic_df.to_csv("/content/drive/My Drive/ICL/FYP/Data/extrinsic_df.csv")